# Project: Predicting Diabetes Onset FHIR EHR Data
### Notebook: 03_Feature_Engineering.ipynb
### Purpose: Develop features for modeling

In [5]:
# import libraries
import pandas as pd
import numpy as np
from typing import List, Tuple
